In [26]:
import cx_Oracle
import sqlalchemy as sa
import pandas as pd
import getpass as gp
import warnings
warnings.filterwarnings("ignore")

dsn_tns = cx_Oracle.makedsn('exa02-scan', '1521', service_name='FBODSPRD')
conn = cx_Oracle.connect(user=r'FB006641', password=gp.getpass("Password"), dsn=dsn_tns)

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import f1_score
from sklearn import metrics
from pprint import pprint
from subprocess import check_call
from sklearn.tree import export_graphviz
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.tree.export import export_text
from sklearn.tree import _tree
import math
import numpy as np
import featuretools as ft
from fancyimpute import KNN
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
import os
import pathlib as pl

Password········


In [27]:
df = pd.read_sql("SELECT * FROM TRAINCEK WHERE IDENTITYTYPE = 'T' AND SUBPROFITCENTER = 'T3'  AND SIMDATE < 20190801",conn)

In [28]:
df["CEKDURUMU"] = df["CEKDURUMU"].map({"Tahsil Iade":0,"Karsiliksiz":1})

In [61]:
def Feature(df,target,depth = 3,min_node_size = 100):
    
    

    class_report = {}
    dot_file = ""
    png_file = ""

    threshold = {}
    thres = [] 
    i = 1
    result = pd.DataFrame(columns = ['Feature', 'Min Value', 'Max Value', 'Count of Class 1' , 'Count of Class 0', 'Total Count', 'Distribution of Class 1'])
    
    if not os.path.isdir("png_files"):        
        os.mkdir("png_files")
        
    if not os.path.isdir("dot_files"):    
        os.mkdir("dot_files")
    
    for column in df.columns:
        if df[column].dtype == 'O':
            df = df.drop(column, axis = 1)
    
    col_names = list(df.drop(target, axis = 1).columns)
    
    for column in col_names:
        temp_df = df[[column,target]]
        temp_df = temp_df.dropna()

        X = temp_df.drop(target,axis=1)
        y = temp_df[target]

        X_train, X_test, y_train, y_test = train_test_split( 
              X, y, test_size = 0.3, random_state = 100)

        temp_X_train = X_train[column]
        temp_X_test = X_test[column]
        temp_X_train = temp_X_train.dropna()
        temp_X_test = temp_X_test.dropna()


        dt = DecisionTreeClassifier(max_depth=depth, random_state=42, min_samples_leaf = min_node_size)

        dt.fit(temp_X_train.values.reshape(-1,1), y_train)
        y_pred = dt.predict(temp_X_test.values.reshape(-1,1))
        class_report[column] = classification_report(y_test, y_pred,output_dict = True)



        dot_file = "dot_files/" + column + ".dot"
        export_graphviz(dt, dot_file , class_names=True, filled = True, proportion = True)

        png_file = "png_files/" + column + ".png"
        check_call(['dot','-Tpng', dot_file, '-o',png_file])

        dot_file = ""
        png_file = ""

        thres.append(temp_X_train.min())

        for i in sorted([a for a in dt.tree_.threshold if a != -2]): 
            thres.append(i)
        thres.append(temp_X_train.max())

        threshold[column] = thres
        thres = []
        
        
    
    
    
    for key in threshold:
        i = 1
        while i < len(threshold[key]):        

            result = result.append({
                "Feature":key,
                "Max Value":threshold[key][i],
                "Min Value":threshold[key][i-1],
                "Total Count":df.loc[(df[key] >= threshold[key][i-1]) & 
                             (df[key] <= threshold[key][i]),][target].count(),
                "Count of Class 1":df.loc[(df[key] >= threshold[key][i-1]) & 
                             (df[key] <= threshold[key][i]),][target].sum(),
                "Count of Class 0":df.loc[(df[key] >= threshold[key][i-1]) & 
                             (df[key] <= threshold[key][i]),][target].count()
                -df.loc[(df[key] >= threshold[key][i-1]) & 
                             (df[key] <= threshold[key][i]),][target].sum()

            },ignore_index=True)
            i += 1

        if len(df.loc[df[key].isnull()]) != 0:
            result = result.append({
                "Feature":key,
                "Max Value":"Null",
                "Min Value":"Null",
                "Total Count":df.loc[df[key].isnull()][target].count(),
                "Count of Class 1":df.loc[df[key].isnull()][target].sum(),
                "Count of Class 0":df.loc[df[key].isnull()][target].count()-df.loc[df[key].isnull()][target].sum()

            },ignore_index=True)              
    
    result["Distribution of Class 1"] = (result["Count of Class 1"]/result["Total Count"])*100

    result["Distribution of Class 1"] = result["Distribution of Class 1"].astype("float")

    result["Distribution of Class 1"] = round(result["Distribution of Class 1"],2)


    result.columns = result.columns.str.upper()
        
    return result

In [62]:
res = Feature(df,"CEKDURUMU")

In [63]:
res.loc[res["FEATURE"].str.contains("KURUMSALKKBGECIKMESI")]

,FEATURE,MIN VALUE,MAX VALUE,COUNT OF CLASS 1,COUNT OF CLASS 0,TOTAL COUNT,DISTRIBUTION OF CLASS 1
247,KURUMSALKKBGECIKMESI,0,0.5,1345,18298,19643,6.85
248,KURUMSALKKBGECIKMESI,0.5,94.5,20,137,157,12.74
249,KURUMSALKKBGECIKMESI,94.5,316,19,160,179,10.61
250,KURUMSALKKBGECIKMESI,316,2619.5,43,176,219,19.63
251,KURUMSALKKBGECIKMESI,2619.5,31432.5,91,158,249,36.55
252,KURUMSALKKBGECIKMESI,31432.5,6.82977e+06,44,120,164,26.83
253,KURUMSALKKBGECIKMESI,Null,Null,420,7621,8041,5.22
